In [ ]:
import requests
import pandas as pd
import os
from io import StringIO
import numpy as np
import csv

In [ ]:
API_TOKEN = ''
data_service_url = 'https://www.ncei.noaa.gov/access/services/data/v1?dataset=local-climatological-data-v2&stations='
date = '&startDate=2018-01-01&endDate=2024-12-31'

headers = {
    'token': API_TOKEN
}

In [ ]:
with open("airport_stations.txt") as f:
    station_list = f.read()
station_list = station_list.split(',')
station_list

In [ ]:
unique_airport = pd.read_csv("unique_airport.csv")

In [ ]:
df = pd.read_csv('ghcnh-station-list.csv', header=None,usecols=[0,1,2,3,4,5,6,7])
final_station = []
for station in station_list:
    if station.split(':')[-1] in list(df[0]):
        final_station.append(station.split(':')[-1])
final_station

In [ ]:
climatological_data = pd.read_csv("climatological_data_2019.csv")
climatological_data['Station'] = climatological_data['Object'].str.split('_').str[1:2].str.join('')
climatological_data

In [ ]:
airport_dir = 'Airport_Weather_Data_Uniq2'

if not os.path.exists(airport_dir):
    os.makedirs(airport_dir)
    
# airport_station = pd.DataFrame(columns=['Station','Name'])

In [ ]:
failed_list = []
checked_list = []


In [ ]:
manual_list = []

In [ ]:

count = 355
for stationid in final_station:
    try:
        if not os.path.isfile(os.path.join(airport_dir,stationid+'.csv')): # skip already existing files
            response_2 = requests.get(climatological_data.loc[climatological_data['Station']==stationid]['URL'].values[0])
            flight_airport = False


            if response_2.status_code == 200:
                print("Checking..." + str(count))
                count+=1


        #         check if the airport name is in flight airport list
                response2_df = pd.read_csv(StringIO(response_2.content.decode('utf-8')))

                if pd.isna(response2_df['NAME'][0]): #the station does not have a name
                    continue
                airport_name = response2_df['NAME'][0].split(",")[0]
                for index, row in unique_airport.iterrows():
                    s1 = set(row['name'].lower().split(" "))
                    s2 = set(airport_name.lower().split(" "))
                    if s1.issubset(s2) or s2.issubset(s1):
                        flight_airport = True
                        print(airport_name + " FOUND!")
                        unique_airport.drop(index, inplace=True)
                        break
                checked_list.append(stationid)
            else:
                failed_list.append(stationid)
                print(f"Error fetching data for station {stationid}: {response_2.status_code}")



    #     if is an airport in the flight data
            if flight_airport:
                response_1 = requests.get(data_service_url+stationid+date)
                print(response_1.status_code)

                if response_1.status_code == 200:
                    if not (response_1.content and response_2.content):
                        print(f"No 'results' found for station {stationid}. Response: {data}")
                        continue
                    else:
                        response1_df = pd.read_csv(StringIO(response_1.content.decode('utf-8')))
                        #insert station information
                        response1_df['LATITUDE'] = response2_df['LATITUDE'][0]
                        response1_df['LONGITUDE'] = response2_df['LONGITUDE'][0]
                        response1_df['ELEVATION'] = response2_df['ELEVATION'][0]
                        response1_df['NAME'] = response2_df['NAME'][0]
                        cols_to_move = ['STATION','LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME']
                        #move columns to front
                        response1_df = response1_df[cols_to_move + [x for x in response1_df.columns if x not in cols_to_move]]

                        response1_df.to_csv(os.path.join(airport_dir,stationid+'.csv'))
                        airport_station = airport_station.append({'Station':stationid, 'Name':response1_df['NAME'][0]},ignore_index=True)
    except:
        manual_list.append(stationid)